In [ ]:
from time import time
import os
import sys
sys.path.append('../')

from utils.codification_cnn import CNNLayer, NNLayer, ChromosomeCNN, FitnessCNN, FitnessCNNParallel
from utils.datamanager import DataManager
from utils.lr_finder import LRFinder
from time import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fitness_cnn = FitnessCNN()

# dataset params:
data_folder = '../../datasets'
classes = []

# Fitness params
epochs = 15
batch_size = 128
verbose = 1
redu_plat = False
early_stop = 0
warm_up_epochs= 5
base_lr = 0.002
smooth = 0.1
cosine_dec = True
lr_find = True

dataset = 'MRDBI'
dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder)
data = dm.load_data()
print(data[0][0].shape)

fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                   epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                   warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find)

In [ ]:
l1_2 = CNNLayer(86, (3,5), 'leakyreLu', 0.262, 1)
l2_2 = CNNLayer(84, (5,3), 'leakyreLu', 0.319, 1)
l3_2 = CNNLayer(243, (1,3), 'prelu', 0.322, 1)
l4_2 = NNLayer(948, 'sigmoid', 0.467)
l5_2 = NNLayer(780, 'sigmoid', 0.441)
base_model = ChromosomeCNN([l1_2, l2_2, l3_2], [l4_2, l5_2], fitness_cnn)

l1_1 = CNNLayer(32, (3,3), 'relu', 0.2, 1)
l2_1 = CNNLayer(64, (3,3), 'relu', 0.2, 1)
l3_1 = NNLayer(512, 'relu', 0.5)
l4_1 = NNLayer(512, 'relu', 0.5)
simple_model = ChromosomeCNN([l1_1, l2_1], [l3_1, l4_1], fitness_cnn)

models = {'simple': simple_model, 'base':base_model}

#### View how change the LR, TIME and ACC depending on the batch size, training on 75 epochs

In [ ]:
warm_up_epochs= 5
lr_find = False
base_lr = 0.002

fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                   epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                   warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find)

BATCH_SIZES = [32, 64, 128, 256, 512, 1024]
#BATCH_SIZES = [BATCH_SIZES[-i-1] for i in range(len(BATCH_SIZES))]
TIMES = []
SCORES = []
LRTS = []
model_name = 'simple'


for batch_size in BATCH_SIZES:
    fitness_cnn.batch_size = batch_size
    ti = time()
    score = fitness_cnn.calc(models[model_name], fp=32)
    lr = fitness_cnn.learning_rate_base
    print("BATCH SIZE ", batch_size)
    print("SCORE: %0.4f" % score)
    print("ELAPSED TIME %0.3f" % (time() - ti))
    print("LEARNING RATE: %0.5f" % lr)
    TIMES.append(time() - ti)
    SCORES.append(score)
    LRTS.append(lr)

In [ ]:
k = 0
BATCH_SIZES_ = BATCH_SIZES[k::]
SCORES_ = SCORES[k::]
LRTS_ = LRTS[k::]
TIMES_ = TIMES[k::]

plt.plot(BATCH_SIZES_, SCORES_, label='scores')
plt.xlabel('Batch size')
plt.xticks(BATCH_SIZES_)
plt.legend()
plt.grid()
plt.show()

plt.plot(BATCH_SIZES_, LRTS_, label='learning rate')
plt.xlabel('Batch size')
plt.xticks(BATCH_SIZES_)
plt.legend()
plt.grid()
plt.show()

plt.plot(BATCH_SIZES_, TIMES_, label="times")
plt.xlabel('Batch size')
plt.xticks(BATCH_SIZES_)
plt.legend()
plt.grid()
plt.show()

#### View how change the LR, TIME and ACC depending on the batch size, training until plateus

In [ ]:
# Fitness params
epochs = 75
batch_size = 128
verbose = 1
redu_plat = False
early_stop = 15
warm_up_epochs = 5
base_lr = 0.002
smooth = 0.1
cosine_dec = True
lr_find = True 

dataset = 'MBI'
dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder)
data = dm.load_data()
print(data[0][0].shape)

fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                   epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                   warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find)

#### Measure training time to reach a given acc %

In [ ]:
from time import time
import keras

class StopAtGoal(keras.callbacks.Callback):
    def __init__(self, goal, **kwargs):
        super().__init__(**kwargs)
        self.goal = goal
        self.epochs = 0
        self.ti = time()
        self.final_acc = None
        self.total_time = None
        self.total_epochs = None
        
    def on_epoch_end(self, epoch, logs=None):
        acc = logs['val_acc']
        self.epochs += 1
        
        if acc >= self.goal:
            self.model.stop_training = True
            self.final_acc = acc
            self.total_time = time() - self.ti
            self.total_epochs = self.epochs
        return
        

In [ ]:
from utils.utils import WarmUpCosineDecayScheduler
from keras.callbacks import ModelCheckpoint

class Fitness_callback(FitnessCNN):
    def set_callbacks(self, file_model=None):
        callbacks = []
        # Create the Learning rate scheduler.
        total_steps = int(self.epochs * self.y_train.shape[0] / self.batch_size)
        warm_up_steps = int(self.warmup_epochs * self.y_train.shape[0] / self.batch_size)
        base_steps = total_steps * (not self.cosine_decay)
        warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=self.learning_rate_base,
                                                total_steps=total_steps,
                                                warmup_learning_rate=0.0,
                                                warmup_steps=warm_up_steps,
                                                hold_base_rate_steps=base_steps)
        callbacks.append(warm_up_lr)

        if file_model is not None:
            checkpoint_acc = ModelCheckpoint(file_model, monitor='val_acc', save_best_only=True)
            callbacks.append(checkpoint_acc)

        if self.early_stop > 0 and keras.__version__ == '2.2.4':
            callbacks.append(EarlyStopping(monitor='val_acc', patience=self.early_stop, restore_best_weights=True))
        elif self.early_stop > 0:
            callbacks.append(EarlyStopping(monitor='val_acc', patience=self.early_stop))
        if self.reduce_plateu:
            callbacks.append(ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                                               patience=5, verbose=self.verb))
        self.stopGoal = StopAtGoal(0.9)
        callbacks.append(self.stopGoal)
        return callbacks

In [ ]:
fitness_cnn = Fitness_callback()

# Fitness params
epochs = 75
batch_size = 128
verbose = 1
redu_plat = False
early_stop = 0
warm_up_epochs= 0
base_lr = 0.001
smooth = 0.1
cosine_dec = True
lr_find = True

fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                   epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                   warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find)

In [ ]:
ti = time()
score = fitness_cnn.calc(base_model, fp=32, test=True)
print("SCORE: %0.4f" % score)
print("ELAPSED TIME %0.3f" % (time() - ti))

stopGoal = fitness_cnn.stopGoal
print("Model reached at goal of %0.2f with %0.2f!" % (stopGoal.goal, stopGoal.final_acc))
print("Total time %0.3f [s] in %d epochs" % (stopGoal.total_time, stopGoal.total_epochs))

BATCH_SIZES = [32, 64, 128, 256, 512, 1024]
TIMES = []
SCORES = []
LRTS = []
EPOCHS = []
model_name = 'base'


for batch_size in BATCH_SIZES:
    fitness_cnn.batch_size = batch_size
    ti = time()
    score = fitness_cnn.calc(models[model_name], fp=32, test=True)
    
    stopGoal = fitness_cnn.stopGoal
    final_acc = stopGoal.final_acc
    total_time = time() - ti
    epochs = stopGoal.total_epochs
    lr = fitness_cnn.learning_rate_base
    
    print("BATCH SIZE ", batch_size)
    print("SCORE: %0.4f" % score)
    print("ELAPSED TIME %0.3f" % total_time)
    print("LEARNING RATE: %0.5f" % lr)
    TIMES.append(total_time)
    SCORES.append(score)
    LRTS.append(lr)
    EPOCHS.append(epochs)

In [ ]:
plt.plot(BATCH_SIZES, TIMES, label="times")
plt.plot(BATCH_SIZES, SCORES, label='scores')
plt.plot(BATCH_SIZES, LRTS, label='learning rate')
plt.plot(BATCH_SIZES, EPOCHS, label='epochs')
plt.xlabel('Batch size')
plt.xscale('log')
plt.show()